In [1]:
import pandas as pd
import numpy as np

In [2]:
TRAIN_RATIO = 0.85
VALIDATION_RATIO = 0.15

In [ ]:
df = pd.read_csv('../data/interim/credit_card_transactions-ibm_v2.csv')
df.head(3)

In [ ]:
df.set_index('Timestamp', inplace=True)
df.index.names=['Timestamp']
df = df.sort_index() # sort index
df.head(3)

In [ ]:
df.tail(3)

In [53]:
df_train = df[(df['Year'] >= 2015) & (df['Year'] < 2019)]
df_train = df[df['Year'] < 2019]
df_test = df[df['Year'] == 2019]

In [54]:
df_train.shape

(22326462, 18)

In [55]:
int(len(df_train) * TRAIN_RATIO)

18977492

In [56]:
train_split_idx = int(len(df_train) * TRAIN_RATIO)

df_val = df_train.iloc[train_split_idx:]
df_train = df_train.iloc[:train_split_idx]


In [57]:
train_split_idx

18977492

In [58]:
df_train.shape

(18977492, 18)

In [59]:
df_val.shape

(3348970, 18)

In [60]:
df_test.shape

(1723938, 18)

In [61]:
df_train.describe()

,User,Card,Year,Month,Day,Amount,Merchant Name,Zip,MCC,Is Fraud?,Hour,Minute
count,1.897749e+07,1.897749e+07,1.897749e+07,1.897749e+07,1.897749e+07,1.897749e+07,1.897749e+07,1.681326e+07,1.897749e+07,1.897749e+07,1.897749e+07,1.897749e+07
mean,9.996312e+02,1.374690e+00,2.010192e+03,6.585566e+00,1.569994e+01,4.387891e+01,-4.728158e+17,5.095376e+04,5.561841e+03,1.313345e-03,1.241988e+01,2.958931e+01
std,5.690579e+02,1.418654e+00,4.386127e+00,3.457822e+00,8.791837e+00,8.253798e+01,4.757016e+18,2.939220e+04,8.788533e+02,3.621630e-02,5.061833e+00,1.731732e+01
min,0.000000e+00,0.000000e+00,1.991000e+03,1.000000e+00,1.000000e+00,-5.000000e+02,-9.222899e+18,5.010000e+02,1.711000e+03,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.090000e+02,0.000000e+00,2.007000e+03,4.000000e+00,8.000000e+00,9.250000e+00,-4.500543e+18,2.838200e+04,5.300000e+03,0.000000e+00,9.000000e+00,1.500000e+01
50%,1.006000e+03,1.000000e+00,2.011000e+03,7.000000e+00,1.600000e+01,3.041000e+01,-7.516291e+17,4.672500e+04,5.499000e+03,0.000000e+00,1.200000e+01,3.000000e+01
75%,1.476000e+03,2.000000e+00,2.014000e+03,1.000000e+01,2.300000e+01,6.583000e+01,3.178683e+18,7.756400e+04,5.812000e+03,0.000000e+00,1.600000e+01,4.500000e+01
max,1.998000e+03,8.000000e+00,2.017000e+03,1.200000e+01,3.100000e+01,1.239050e+04,9.223292e+18,9.992800e+04,9.402000e+03,1.000000e+00,2.300000e+01,5.900000e+01


In [62]:
df_val.describe()

,User,Card,Year,Month,Day,Amount,Merchant Name,Zip,MCC,Is Fraud?,Hour,Minute
count,3.348970e+06,3.348970e+06,3.348970e+06,3.348970e+06,3.348970e+06,3.348970e+06,3.348970e+06,2.908471e+06,3.348970e+06,3.348970e+06,3.348970e+06,3.348970e+06
mean,1.006820e+03,1.277134e+00,2.017514e+03,6.687433e+00,1.587203e+01,4.286772e+01,-4.911627e+17,5.091436e+04,5.558139e+03,8.199536e-04,1.239644e+01,2.958500e+01
std,5.705019e+02,1.364165e+00,4.998020e-01,3.363490e+00,8.825480e+00,8.064359e+01,4.764657e+18,2.938222e+04,8.811407e+02,2.862309e-02,5.078588e+00,1.732034e+01
min,0.000000e+00,0.000000e+00,2.017000e+03,1.000000e+00,1.000000e+00,-5.000000e+02,-9.222899e+18,1.001000e+03,1.711000e+03,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.170000e+02,0.000000e+00,2.017000e+03,4.000000e+00,8.000000e+00,9.040000e+00,-4.540582e+18,2.821600e+04,5.300000e+03,0.000000e+00,9.000000e+00,1.500000e+01
50%,1.012000e+03,1.000000e+00,2.018000e+03,7.000000e+00,1.600000e+01,2.924000e+01,-8.138808e+17,4.715000e+04,5.499000e+03,0.000000e+00,1.200000e+01,3.000000e+01
75%,1.483000e+03,2.000000e+00,2.018000e+03,1.000000e+01,2.400000e+01,6.300000e+01,3.189517e+18,7.751100e+04,5.812000e+03,0.000000e+00,1.600000e+01,4.500000e+01
max,1.999000e+03,8.000000e+00,2.018000e+03,1.200000e+01,3.100000e+01,5.682220e+03,9.222877e+18,9.992800e+04,9.402000e+03,1.000000e+00,2.300000e+01,5.900000e+01


In [63]:
df_test.describe()

,User,Card,Year,Month,Day,Amount,Merchant Name,Zip,MCC,Is Fraud?,Hour,Minute
count,1.723938e+06,1.723938e+06,1723938.0,1.723938e+06,1.723938e+06,1.723938e+06,1.723938e+06,1.495120e+06,1.723938e+06,1.723938e+06,1.723938e+06,1.723938e+06
mean,1.004711e+03,1.276298e+00,2019.0,6.525915e+00,1.571612e+01,4.274106e+01,-4.955698e+17,5.103297e+04,5.559918e+03,1.210600e-03,1.238867e+01,2.956494e+01
std,5.702081e+02,1.367541e+00,0.0,3.446948e+00,8.795041e+00,8.012487e+01,4.765159e+18,2.944270e+04,8.817533e+02,3.477263e-02,5.077021e+00,1.732823e+01
min,0.000000e+00,0.000000e+00,2019.0,1.000000e+00,1.000000e+00,-5.000000e+02,-9.222899e+18,1.001000e+03,1.711000e+03,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.170000e+02,0.000000e+00,2019.0,4.000000e+00,8.000000e+00,9.010000e+00,-4.536641e+18,2.830300e+04,5.300000e+03,0.000000e+00,9.000000e+00,1.500000e+01
50%,1.009000e+03,1.000000e+00,2019.0,7.000000e+00,1.600000e+01,2.921000e+01,-8.981400e+17,4.736200e+04,5.499000e+03,0.000000e+00,1.200000e+01,3.000000e+01
75%,1.482000e+03,2.000000e+00,2019.0,1.000000e+01,2.300000e+01,6.298000e+01,3.189517e+18,7.757300e+04,5.812000e+03,0.000000e+00,1.600000e+01,4.500000e+01
max,1.999000e+03,8.000000e+00,2019.0,1.200000e+01,3.100000e+01,6.613440e+03,9.222877e+18,9.982900e+04,9.402000e+03,1.000000e+00,2.300000e+01,5.900000e+01


In [64]:
df_train['Is Fraud?'].mean(), df_val['Is Fraud?'].mean(), df_test['Is Fraud?'].mean()

(0.0013133453040053975, 0.0008199535976733145, 0.0012106003812202063)

In [65]:
df_train.to_csv('../data/processed/train.csv',
              header=df_train.columns,
              index=False)

In [66]:
df_val.to_csv('../data/processed/val.csv',
              header=df_val.columns,
              index=False)

In [67]:
df_test.to_csv('../data/processed/test.csv',
              header=df_test.columns,
              index=False)